###### 虽然复现AES这个版本的加解密脚本多少也花了我三四天时间，但是由于本质上没有太多难磕的陌生知识，最多就是在伽罗瓦域里面的一些运算规则和数论内矩阵相关乘法花了点时间去理解和复现代码，其他时间基本上是操作列表，偶尔给值加些运算符之类的，所以记录也就没有以前那么详细，理论上实现AES可能稍微复杂，但是真的不难，貌似还有其他版本的，到时候是否会改进脚本和学习深层次还要视需求而定



In [1]:
message = b'hello, tomorrow.'
key = b'GOODBYE, TODAY..'
def getjuzhen(m):
    '''排列成字节矩阵'''
    juzheng = []
    M=[]
    for i in range(len(m)):
        if (i+1)%4==0:
            M.append(m[i])
            juzheng.append(M)
            M = []
        else:
            M.append(m[i]) 

    return juzheng
       
def addroundkey(m,k):
    '''轮密钥加'''
    mm = []
    kk = []
    r = []
    for i in range(4):
        rr = []
        for j in range(4):
            rr.append(m[i][j]^k[i][j])
        r.append(rr)
    return r

def ext_gcd(a, b):    
    if b == 0:          
        return 1, 0, a     
    else:         
        x, y, gcd = ext_gcd(b, a % b) #递归直至余数等于0(需多递归一层用来判断)        
        x, y = y, (x - (a // b) * y) #辗转相除法反向推导每层a、b的因子使得gcd(a,b)=ax+by成立         
        return x, y, gcd

def xtime(binnumber1):
    '''GF域内的乘法算法之一'''
    flag = 1
    m = 0b100011011
    m1 = 0b00011011
    # ll2 = list(bin(binnumber2)[2:])
    ll1 = list(bin(binnumber1)[2:].zfill(8))
    p = len(ll1)
    # print(ll1)
    if ll1[0]=='0':
        return binnumber1<<1
    else:
        # print("?")
        return (binnumber1<<1)^m

def times(b1,b2):
    '''GF域内的乘法'''
    ll2 = list(bin(b2)[2:].zfill(8))
    L = []
    vv = b1
    loc = len(ll2)-ll2.index('1')
    # print(loc)
    for i in range(loc):
        L.append(vv)
        vv = xtime(vv)
    r = 0
    j = 0
    # print(L)
    lm = ll2[-1::-1]
    for i in range(len(lm)):
        if lm[i]=='1':
            # print(L[i])
            r^=L[i]

    return r

def liancheng(l):
    '''GF内3的幂'''
    if l ==0:
        return 1
    r = 3
    for i in range(l-1):
        r = times(r, 3)
    return r

def getre1():
    '''生成元查表求逆'''
    re1={}
    re1[0]=1
    for i in range(1,255):
        re1[i]=liancheng(i)
    re1[255]=1
    return re1
re1 = getre1()

def getre2():
    '''生成逆表'''
    re2={}
    re2[1]=0
    for i in range(1,255):
        re2[liancheng(i)]=i
    return re2
re2 = getre2()

def findinvert(b,re1,re2):
    if b == 0:
        return 0
    lll = re2[b]
    re = re1[255-lll]
    return re

def getsbox(b):
    pl = bin(findinvert(b, re1, re2))[2:].zfill(8)
    pppl = []
    for i in pl:
        pppl.append(int(i))
    ppl = pppl[-1::-1]
    m = []
    c1 = (1*ppl[0]) ^ (0*ppl[1]) ^ (0*ppl[2]) ^ (0*ppl[3]) ^ (1*ppl[4]) ^ (1*ppl[5]) ^ (1*ppl[6]) ^ (1*ppl[7]) ^ 1 
    c2 = (1*ppl[0]) ^ (1*ppl[1]) ^ (0*ppl[2]) ^ (0*ppl[3]) ^ (0*ppl[4]) ^ (1*ppl[5]) ^ (1*ppl[6]) ^ (1*ppl[7]) ^ 1 
    c3 = (1*ppl[0]) ^ (1*ppl[1]) ^ (1*ppl[2]) ^ (0*ppl[3]) ^ (0*ppl[4]) ^ (0*ppl[5]) ^ (1*ppl[6]) ^ (1*ppl[7]) ^ 0 
    c4 = (1*ppl[0]) ^ (1*ppl[1]) ^ (1*ppl[2]) ^ (1*ppl[3]) ^ (0*ppl[4]) ^ (0*ppl[5]) ^ (0*ppl[6]) ^ (1*ppl[7]) ^ 0 
    c5 = (1*ppl[0]) ^ (1*ppl[1]) ^ (1*ppl[2]) ^ (1*ppl[3]) ^ (1*ppl[4]) ^ (0*ppl[5]) ^ (0*ppl[6]) ^ (0*ppl[7]) ^ 0 
    c6 = (0*ppl[0]) ^ (1*ppl[1]) ^ (1*ppl[2]) ^ (1*ppl[3]) ^ (1*ppl[4]) ^ (1*ppl[5]) ^ (0*ppl[6]) ^ (0*ppl[7]) ^ 1 
    c7 = (0*ppl[0]) ^ (0*ppl[1]) ^ (1*ppl[2]) ^ (1*ppl[3]) ^ (1*ppl[4]) ^ (1*ppl[5]) ^ (1*ppl[6]) ^ (0*ppl[7]) ^ 1 
    c8 = (0*ppl[0]) ^ (0*ppl[1]) ^ (0*ppl[2]) ^ (1*ppl[3]) ^ (1*ppl[4]) ^ (1*ppl[5]) ^ (1*ppl[6]) ^ (1*ppl[7]) ^ 0 
    m.append(c1)
    m.append(c2)
    m.append(c3)
    m.append(c4)
    m.append(c5)
    m.append(c6)
    m.append(c7)
    m.append(c8)
    bbb = ''.join(str(i) for i in m[-1::-1])
    # return hex(int(bbb,2))
    return int(bbb,2)

def sbox():
    sbox = {}
    for i in range(256):
        sbox[i]=getsbox(i)
    return sbox
# success!!!
def ByteSubstitution(byte):
    return sbox()[byte]


jz1 = [[2, 1,1,3], [3,2,1,1], [1,3,2,1], [1,1,3,2]]
jz2 = [[0xD4,0xBF,0x5D,0x30], [0x25,0x25,0x25,0x25], [0x25,0x25,0x25,0x25], [0x25,0x25,0x25,0x25]]
def jztimes(jz1,jz2):
    '''GF域下的矩阵乘法'''
    r = []
    for j in range(4):
        rr = []
        for i in range(4):
            rr.append((times(jz1[0][i], jz2[j][0]) ^ times(jz1[1][i], jz2[j][1]) ^ times(jz1[2][i], jz2[j][2]) ^ times(jz1[3][i],jz2[j][3])))
        r.append(rr)

    return r
    
def MC(jz):
    '''列混合'''
    jz1 = [[2, 1,1,3], [3,2,1,1], [1,3,2,1], [1,1,3,2]]
    return jztimes(jz1, jz)

In [3]:
def BS(jz):
    '''字节代换层'''
    r=[]
    for i in range(4):
        rr = []
        for j in range(4):
            rr.append(ByteSubstitution(jz[i][j]))
        r.append(rr)
    return    r

def wy(jz,j):
    '''行间位移'''
    nn = jz[:]
    rr = []
    for i in range(4):
            x = jz[(i+j)%4][:]
            y = nn[i%4][:]
            y[j]=x[j]
            
            rr.append(y)
    return rr

def SR(jz):
    '''行位移'''
    r = jz
    for i in range(1,4):
        r = wy(r, i)
        
    return r   

def danci(jz,k):
    '''一轮加密'''
    n1 = BS(jz)
    n2 = SR(n1)
    n3 = MC(n2)
    r = addroundkey(n3, k)
    return r
    
def G(k,time):
    '''G函数处理密钥'''
    nn = k[:]
    for i in range(4):
        x = k[(i+1)%4]
        nn[i%4]=x
    rr = nn[:]
    # print(rr)
    for i in range(4):
        rr[i]=sbox()[nn[i]]
    Rcon = [ 0X00,0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1B, 0x36]
    rr[0]=rr[0]^Rcon[time]

    return rr

def oncekeywhitening(key,time):
    '''一次密钥漂白'''
    r = []
    for i in range(4):
        rr = []
        for j in range(4):
            if i==0:
                rr.append(key[i][j]^G(key[3], time)[j])
            else:
                rr.append(key[i][j]^r[i-1][j])
        r.append(rr)
    return r

def KW(key):
    '''生成密钥列表'''
    keys = [key]
    
    for i in range(1,11):
        keys.append(oncekeywhitening(keys[i-1], i))
    return keys

def AES(m,k):
    keys = KW(k)
    c = addroundkey(m, keys[0])
    # print(BS(c))
    for i in range(1,10):
        c=danci(c, keys[i])
    c = BS(c)
    c=SR(c)
    c = addroundkey(c, keys[10])
    return c


In [4]:
m = getjuzhen(message)

k = getjuzhen(key)

In [5]:
# 现在开始解密脚本
# sbox()
# {i:j for j,i in sbox().items()}

def inv_sbox():
    return {i:j for j,i in sbox().items()}
# inv_box()

def inv_BS(jz):
    r=[]
    for i in range(4):
        rr = []
        for j in range(4):
            rr.append(inv_ByteSubstitution(jz[i][j]))
        r.append(rr)
    return    r

def inv_ByteSubstitution(byte):
    return inv_sbox()[byte]
# inv_ByteSubstitution(0xdd)

def inv_addroundkey(m, k):
    return addroundkey(m, k)

def inv_SR(jz):
    for i in range(3):
        jz = SR(jz)
    return jz

def inv_MC(jz):
    jz2 = [[14, 9, 13, 11], [11, 14, 9, 13], [13, 11, 14, 9], [9, 13, 11, 14]]
    return jztimes(jz2, jz)

def decryptAES(c,k):
    print('aaa')
    state = getjuzhen(c)
    keys = KW(getjuzhen(k))
    # print(keys[-1])
    state = inv_addroundkey(state, keys[-1])
    state = inv_SR(state)
    state = inv_BS(state)
    
    for i in range(1,10):
        state = inv_addroundkey(state, keys[10-i])
        state = inv_MC(state)
        state = inv_SR(state)
        state = inv_BS(state)
    state = inv_addroundkey(state, keys[0])
    return state

def bytes2matrix(text):
    """ Converts a 16-byte array into a 4x4 matrix.  """
    return [list(text[i:i+4]) for i in range(0, len(text), 4)]

def matrix2bytes(matrix):
    """ Converts a 4x4 matrix into a 16-byte array.  """
    # ????
    r = b''
    for i in matrix:
        r+=bytes(i)

    return r
c = matrix2bytes(AES(m, k))
key

b'GOODBYE, TODAY..'

In [6]:
decryptAES(c, key)


aaa


[[104, 101, 108, 108],
 [111, 44, 32, 116],
 [111, 109, 111, 114],
 [114, 111, 119, 46]]

In [7]:
key

b'GOODBYE, TODAY..'

In [10]:
key

b'\xc3,\\\xa6\xb5\x80^\x0c\xdb\x8d\xa5z*\xb6\xfe\\'

In [12]:
key

b'GOODBYE, TODAY..'

In [9]:
from Crypto.Cipher import AES
cryptor = AES.new(key, AES.MODE_CBC,key)
c = cryptor.encrypt(message)
c

b'vQ\xe7\x9eq\xc3\x00\x9d.fsT{\xbb\x1cV'

In [ ]:
#https://zhuanlan.zhihu.com/p/41716899
#https://bbs.pediy.com/thread-253884.htm#msg_header_h1_8

#https://blog.csdn.net/sinat_36329095/article/details/106976277?utm_medium=distribute.pc_aggpage_search_result.none-task-blog-2~aggregatepage~first_rank_ecpm_v1~rank_v31_ecpm-1-106976277.pc_agg_new_rank&utm_term=aes%E9%87%8C%E9%9D%A2xtime%28%29%E8%BF%90%E7%AE%97&spm=1000.2123.3001.4430
#https://blog.csdn.net/lplp9822/article/details/80906583

#https://blog.csdn.net/zmnbehappy/article/details/78469850
#https://zhuanlan.zhihu.com/p/41648498、


In [ ]:
#加密脚本成功了，